In [1]:
import tensorflow as tf
# Load and convert a frozen graph
graph_def = tf.GraphDef()
with tf.gfile.GFile("chest_model.pb", 'rb') as f:
    graph_def.ParseFromString(f.read())

In [4]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image
import numpy as np
import os
files = os.listdir('chest_xray/test/NORMAL/')
img1 = image.load_img(r'chest_xray/test/NORMAL/' + files[0], target_size=(299, 299))
img_array1 = image.img_to_array(img1)
img_array_expanded_dims1 = np.expand_dims(img_array1, axis=0)
input_img = preprocess_input(img_array_expanded_dims1)


converter = trt.TrtGraphConverter(
    input_graph_def=graph_def,
    nodes_blacklist=['dense_1/Sigmoid'],
    precision_mode='INT8',
    use_calibration=True)
frozen_graph = converter.convert()


In [5]:
frozen_graph = converter.calibrate(
    fetch_names=['dense_1/Sigmoid'],
    num_runs=1,
    feed_dict_fn=lambda: {'input_1:0': input_img})


In [8]:
from tensorflow.python.platform import gfile
with gfile.FastGFile("chest_TensorRT_model_88.pb", 'wb') as f:
    f.write(frozen_graph.SerializeToString())